In [6]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from model import ConditionalModel
from ema import EMA
import torch.optim as optim

In [7]:
import pathlib
import pandas as pd
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [10]:
def create_input_curves(folder, curv_num=0):
    one_mode_ds = []
    lbls_ds = []
    
    for file in os.listdir(os.path.join(folder)):
        
        df = pd.read_csv(folder+file, header=None)
        
        # curve_data = []
        # for i in range(5):
        # curve_data.append((df[4][curv_num:500+curv_num:5]).to_numpy())
        
        one_mode_ds.append(((df[5][curv_num:500+curv_num:5])/250000).to_numpy())
        lbls_ds.append(np.array([df[0][0], df[1][0], df[2][0]]))
        
        
    return np.array(one_mode_ds), np.array(lbls_ds)

In [9]:
data_folder = 'D:/ML_DL_AI_stuff/!DispCurves_Proj/disp_data/'

# each file in datafolder contains data 5 modes (to the moment). see 'plot_separate_disp_curv' script
# but for the first time I want tot usnly first modes
n_modes = 5 
main_folder = 'for_NN_sorted/'
train_folder = main_folder+'train/'
valid_folder = main_folder+'valid/'
test_folder = main_folder+'test/'

In [19]:
dataset, labels = create_input_curves(data_folder+train_folder)
# convert numpy arrays to PyTorch tensors
dataset = dataset.astype(np.float32)
labels = labels.astype(np.float32)
# dataset = torch.from_numpy(dataset)# .float()
# labels = torch.from_numpy(labels) # .float()

In [20]:
# Define the shape vector and dispersion curve dataset
shape_vectors = labels # your shape vector data
dispersion_curves = dataset # your dispersion curve data
dataset = TensorDataset(torch.tensor(shape_vectors), torch.tensor(dispersion_curves))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [25]:
dispersion_curves.shape

torch.Size([128, 100])

In [22]:
# Define the model

# , input_size=3, output_size=100
model = ConditionalModel(num_steps) # assuming shape vector is 3 dim and dispersion curve is 100 dim
optimizer = optim.Adam(model.parameters(), lr=1e-3)
ema = EMA(0.9)
ema.register(model)

# Define the loss function
def dispersion_curve_loss(model, x, y):
    generated_curve = model(x, y)
    return torch.mean(torch.abs(generated_curve - y))

# Train the model
for t in range(1000):
    for shape_vectors, dispersion_curves in dataloader:
        # Compute the loss
        loss = dispersion_curve_loss(model, shape_vectors, dispersion_curves)

        # Perform backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # update the exponential moving average
        ema.update(model)
        
        # Print loss
        if (t % 100 == 0):
            print("Loss at step ", t, ": ", loss.item())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x3 and 2x128)

In [ ]:
# Save the model
torch.save(model.state_dict(), "dispersion_curve_model.pth")
print("Training complete.")

# Test the model
shape_vectors_test, dispersion_curves_test = generate_test_data(num_test_samples)
dispersion_curves_pred = model(shape_vectors_test)
print("Mean squared error on test data: ", mean_squared_error(dispersion_curves_test, dispersion_curves_pred))

# first time 

In [1]:
import torch
from model import ConditionalModel
from ema import EMA
import torch.optim as optim

In [3]:
num_steps = 100

In [4]:
# Define the model
model = ConditionalModel(num_steps)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Create EMA model
ema = EMA(0.9)
ema.register(model)

# Batch size
batch_size = 128

In [ ]:
# Loop through training steps
for t in range(1000):
    # X is a torch Variable
    permutation = torch.randperm(dataset.size()[0])
    for i in range(0, dataset.size()[0], batch_size):
        # Retrieve current batch
        indices = permutation[i:i+batch_size]
        shape_vectors = dataset[indices][:, 0:3]
        disp_curves = dataset[indices][:, 3:]
        # Compute the loss.
        loss = noise_estimation_loss(model, shape_vectors, disp_curves, alphas_bar_sqrt, one_minus_alphas_bar_sqrt, num_steps)
        # Before the backward pass, zero all of the network gradients
        optimizer.zero_grad()
        # Backward pass: compute gradient of the loss with respect to parameters
        loss.backward()
        # Perform gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
        # Calling the step

# second time

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return len(self.data)

In [ ]:
class ConditionalModel(nn.Module):
    def __init__(self, num_steps, shape_size):
        super(ConditionalModel, self).__init__()
        self.shape_size = shape_size
        self.num_steps = num_steps
        self.hidden_size = 64
        self.shape_input = nn.Linear(shape_size, self.hidden_size)  # new input layer
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, num_layers=1, batch_first=True)
        self.fc_mu = nn.Linear(self

# questions

In [ ]:
do you know github repo  azad-academy by J. Rafid Siddiqui?

In [ ]:
this github has a repository named denoising-diffusion-model? which contains the model which I described you before

In [ ]:
i need to train neural network to generate dispersion curve by given shape of the element of periodic structure. Shape I define as a vecor of 3 values. I thought that shape is a label and dispersion curve is a data array that should be restored from noise using trained diffusion model
and I want to train diffusion model to solve this task